In [40]:
import operator
import os
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import tools_condition
from langgraph.types import Command
from langgraph.graph import MessagesState
from pydantic import BaseModel, Field
from typing import Annotated, Sequence, Optional, Dict, Any, TypedDict, Literal
from langchain_core.messages import BaseMessage, AIMessage
from langchain_core.prompts import PromptTemplate

### Initializing the LLMs

In [17]:
## Langchain and Langsmith tracing
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ["LANGCHAIN_TRACING_V2"]="true"

## Getting Froq API key
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

# EMBEDDING_MODEL = "BAAI/bge-small-en"
GROQ_INFERENCE_MODEL = "deepseek-r1-distill-llama-70b"
OPENAI_INFERENCE_MODEL = "gpt-4o"
# INFERENCE_MODEL = "gemma2-9b-it"

In [18]:
llm = ChatGroq(model=GROQ_INFERENCE_MODEL, temperature=0.3)
print('--------Creating the response from llm based on contexts--------')

# llm = ChatOpenAI(model=OPENAI_INFERENCE_MODEL, temperature=0.3)
# print('--------Creating the response from llm based on contexts--------')

--------Creating the response from llm based on contexts--------


### State of the Agents

In [ ]:
class SupervisorState(MessagesState):
    """State of the Supervisor Agent"""
    query: str
    research_results: Optional[Dict[str, Any]] = None
    summary_results: Optional[Dict[str, Any]] = None
    final_documents: Optional[str] = None
    next: Optional[Literal['research', 'report', 'FINISH']] = None
    reasoning: Optional[str] = None
    task_details: Optional[str] = None


In [23]:
class ResearchState(BaseModel):
    """State of the Team 1: Research Agent"""
    messages: Annotated[Sequence[BaseMessage], operator.add]
    research_query: str
    medical_results: Optional[Dict[str, Any]]
    financial_results: Optional[Dict[str, Any]]

In [24]:
class ReportingState(BaseModel):
    """State of the Team 2: Reporting Agent"""
    messages: Annotated[Sequence[BaseMessage], operator.add]
    research_data: Dict[str, Any]
    summary: Optional[str]
    final_report: Optional[str]

### Router of the Agents

In [38]:
class SupervisorRouter(TypedDict):
    next: Literal['research', 'report', 'FINISH']
    reasoning: str
    task_details: str

In [28]:
class ResearchRouter(TypedDict):
    next: Literal['medical', 'financial']

In [29]:
class ReportRouter(TypedDict):
    next: Literal['summary', 'document']

### Agent Prompt Template

In [32]:
supervisor_prompt_template = """You are the Supervisor Agent coordinating a hierarchical multi-agent research system.

Your responsibilities:
1. Analyze incoming requests to determine if research or reporting is needed
2. Coordinate between Team 1 (Research) and Team 2 (Reporting)
3. Make decisions on workflow progression

Teams under your supervision:
- Team 1 (Research): Coordinates medical/pharma (Team 3) and financial (Team 4) research
- Team 2 (Reporting): Coordinates summary creation (Team 5) and document generation (Team 6)

Decision criteria:
- If no research results exist and task requires data gathering then route to "research" agent i.e. Team 1
- If research results exist but no report generated then route to "reporting" agent i.e. Team 2  
- If final document is complete then route to "end"

Always respond with your decision and reasoning in JSON format:
{{  "next_action": "research|reporting|end", 
    "reasoning": "explanation", 
    "task_details": "specific instructions"}}

Here is the user query: {question}
"""

### Agent Prompts

In [33]:
supervisor_prompt = PromptTemplate(
    template=supervisor_prompt_template,
    input_variables=["question"]
)

### Agents

#### Supervisor Agent

In [ ]:
def supervisor(state:SupervisorState)->Command[Literal['research', 'report', '__end__']]:
    """This is the supervisor agent. It takes the query and redirects it to either the research node or reporting node"""
    llm_with_structured_output = llm.with_structured_output(SupervisorRouter)
    formatted_prompt = supervisor_prompt_template.format(question=state["query"])
    response = llm_with_structured_output.invoke(formatted_prompt)
    
    goto = response["next"]

    if goto == "FINISH":
        goto=END
    
    supervisor_message = AIMessage(
        content=f"SUPERVISOR DECISION: Routing to {goto}. Reasoning: {response['reasoning']}. Task Details: {response['task_details']}"
    )
    
    return Command(goto=goto, update={
        "query": state.query,
        "next": goto,
        "reasoning": response["reasoning"],
        "task_details": response["task_details"],
        "messages": [supervisor_message]})
    

#### Researcher Agent

In [7]:
def researcher(state:AgentState):
    pass

#### Reporter Agent

In [8]:
def reporter(state:AgentState):
    pass

#### Medical Researcher Agent

In [9]:
def medical_researcher(state:AgentState):
    pass

#### Finance Researcher Agent

In [10]:
def finance_researcher(state:AgentState):
    pass

#### Document Summarizer Agent

In [11]:
def document_summarizer(state:AgentState):
    pass

#### Document Generator Agent

In [12]:
def document_generator(state:AgentState):
    pass

### Workflow

In [14]:
graph_builder = StateGraph(AgentState)
graph_builder.add_node("Researcher", researcher)
graph_builder.add_node("Reporter", reporter)
graph_builder.add_node("Supervisor",supervisor)
graph_builder.add_node("Medical Researcher", medical_researcher)
graph_builder.add_node("Finance Researcher", finance_researcher)
graph_builder.add_node("Document Summarizer", document_summarizer)
graph_builder.add_node("Document Generator", document_generator)
graph_builder.add_edge(START, "Supervisor")
# graph_builder.add_conditional_edges("ai_assistant",
#                                     tools_condition)
# graph_builder.add_edge("tools","ai_assistant")
# app_react = graph_builder.compile(checkpointer=memory, interrupt_before=["tools"])